# Poor Man's Hot Spot

I'm trying to understand what is going on with the Sewer 311 requests.  Before I look at the standard ways to build hot spots (DBSCAN, Getis-Ord Gi*, ...) I need to gain some intuition into what is going on.

This notebook is also used to relate DEM with 311 locations.  Maybe this should be moved?

In [ ]:
brooklyn_311_gdf = gpd.read_parquet('../data/processed/brooklyn/brooklyn-2021-311.parq').reset_index().drop(columns='index')

water_sewer_311_gdf = brooklyn_311_gdf[brooklyn_311_gdf['Complaint Type'].isin(['Water System', 'Sewer'])]

sewer_311_gdf = water_sewer_311_gdf.query(f"`Complaint Type` == 'Sewer'").reset_index().drop(columns='index')

In [ ]:
brooklyn_gdf = gpd.read_parquet('../data/processed/admin-boundaries/brooklyn.parq')

In [ ]:
brooklyn_turfs_gdf = gpd.read_parquet('../data/processed/brooklyn/brooklyn-turfs.parq')

In [ ]:
brooklyn_turfs_subset_gdf = brooklyn_turfs_gdf[['OrgName', 'OrgWebSite', 'PrimST', 'PopID', 'Shape_Area', 'geometry']].copy().to_crs('epsg:2263')

pertinent sounds awkward?

In [ ]:
pertinent_turfs_primst = ['Waterfront / Beach / Shoreline', 
                          'Watershed / Sewershed', 
                          'Stream / River / Canal', 
                          'Salt Marsh', 
                          'Public Right of Way (Sidewalk, street ends, traffic island, public plaza)', 
                          'Freshwater Wetland']

Maybe this one could be renamed to primst_based_turfs_gdf?

In [ ]:
pertinent_turfs_gdf = brooklyn_turfs_subset_gdf.query(f"PrimST in @pertinent_turfs_primst").reset_index().drop(columns='index')

In [ ]:
len(pertinent_turfs_gdf)

In [ ]:
pertinent_turfs_gdf.sort_values('Shape_Area', inplace=True, ascending=False)

In [ ]:
from ipyleaflet import Heatmap

In [ ]:
center = brooklyn_gdf.iloc[0].geometry.centroid.y, brooklyn_gdf.iloc[0].geometry.centroid.x

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True

google_map = TileLayer(
    url="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Maps",
)
google_map.base = True

google_satellite = TileLayer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Satellite"
)
google_satellite.base = True

map_display = Map(center=center, zoom=12,
                  layers=[google_satellite, google_map, imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())

map_display.add_control(FullScreenControl())

map_display

In [ ]:
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in sewer_311_gdf.geometry]
heat_map = Heatmap(locations=heat_data, radius=20, blur=10, name='Heat')

In [ ]:
map_display.add_layer(heat_map)

In [ ]:
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange', 'purple', 'blue']),
    }

import json
import random
#turfs_geojson = brooklyn_turf_subset_gdf.to_json()
turfs_geojson = pertinent_turfs_gdf.iloc[25:55].to_crs('epsg:4326').to_json()

geo_json = GeoJSON(
    data=json.loads(turfs_geojson),
    #style={
    #    'opacity': 0.1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
   # },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.8
    },
    #style_callback=random_color,
    name='turfs geojson'
)

map_display.add_layer(geo_json)

turf_html = HTML('''Hover over a turf''')
turf_html.layout.margin = '0px 20px 20px 20 px'
turf_control = WidgetControl(widget=turf_html, position='bottomright')

def update_turf_html(feature, **kwargs):
    turf_html.value = f"<b>Name: {feature['properties']['OrgName']}<br><b>Primary: {feature['properties']['PrimST']}"
    
map_display.add_control(turf_control)  # does += work for this?

geo_json.on_hover(update_turf_html)

In [ ]:
sewer_311_gdf

In [ ]:
call_density_gdf = GeoDataFrame(sewer_311_gdf.geometry.value_counts().to_frame().reset_index().rename(columns={'index': 'geometry', 'geometry': 'count'}))

In [ ]:
more_than_two_gdf = call_density_gdf[call_density_gdf['count'] > 2].reset_index().drop(columns='index')
more_than_three_gdf = call_density_gdf[call_density_gdf['count'] > 3].reset_index().drop(columns='index')

In [ ]:
more_than_two_gdf['count'].value_counts().sort_index()

In [ ]:
more_than_three_gdf['count'].value_counts()

In [ ]:
more_than_two_gdf['bin'] = pd.cut(more_than_two_gdf['count'], bins=[0, 5, 9, 31])

In [ ]:
more_than_two_gdf['bin'].value_counts()

In [ ]:
more_than_three_gdf['bin'] = pd.cut(more_than_three_gdf['count'], bins=[0, 5, 9, 31], labels=['small', 'medium', 'large'])

In [ ]:
more_than_three_gdf['bin'].value_counts()

In [ ]:
c_map = {'small': '#4E9A26', 'medium': '#EBC621', 'large': '#AC1212'}

In [ ]:
markers = list()

for i, row in tqdm(more_than_three_gdf.iterrows()):
    
    fill_color = c_map[row.bin]
    marker = CircleMarker(location=(row.geometry.y, row.geometry.x), radius=5, stroke=False, fill_color=fill_color, fill_opacity=1.0)
    msg = HTML()
    msg.value = "count: {}".format(row['count'])
    marker.popup = msg
    markers.append(marker)
    more_than_three_gdf.loc[i, 'marker'] = marker

dep_cluster = MarkerCluster(markers=markers, name='311 Call')

map_display.add_layer(LayerGroup(name=f"311", layers=markers))


# DEM

In [ ]:
brooklyn_dem_gdf = gpd.read_parquet('../data/processed/brooklyn/brooklyn-dem.parq')

In [ ]:
len(brooklyn_dem_gdf)

In [ ]:
brooklyn_dem_gdf.columns

https://answerbun.com/

**Note:** This is very computationally intense.  Beware. I saved a version for my purposes.

In [ ]:
from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 
pts3 = brooklyn_dem_gdf.geometry.unary_union
def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
     nearest = brooklyn_dem_gdf.geometry == nearest_points(point, pts)[1]
     return brooklyn_dem_gdf[nearest].elevation.iloc[0]

In [ ]:
more_than_three_gdf['elevation'] = more_than_three_gdf.apply(lambda row: near(row.geometry), axis=1)

In [ ]:
more_than_three_gdf.iloc[27]

In [ ]:
more_than_three_gdf.elevation.isnull().sum()

In [ ]:
more_than_three_gdf.columns

In [ ]:
more_than_three_gdf[['geometry', 'count', 'bin', 'elevation']].to_parquet('../data/processed/brooklyn/brooklyn-311-elevation.parq')